# Suicide Tweet Detection ML Pipeline

Mengunduh atau import library yang dibutuhkan dalam pengerjaan pembuatan ML pipeline.

In [1]:
#import library
import tensorflow_transform as tft
from tensorflow import keras
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## Prepare Dataset

Langkah pertama dalam mempersiapkan dataset adalah dengan melakukan import dataset yang akan diproses menggunakan algoritma machine learning. Pada pengerjaan ini, dataset bersumber pada Kaggle sebagai berikut: (https://www.kaggle.com/datasets/aunanya875/suicidal-tweet-detection-dataset)

Dataset berisi tentang kumpulan postingan pada media sosial Twitter (tweet) dengan penilaian apakah tweet tersebut memiliki makna untuk mengakhiri hidup atau tidak. dataset yang orisinal terdapa pada folder 'data_ori', sedangkan data yang digunakan pada folder 'data' merupakan dataset berformat csv yang telah diolah dengan menghilangkan dua baris dengan null values dan menggunakan categorical encoding untuk pengkategorian makna tweet (0: not suicide post; 1: potential suicide post)

In [2]:
tweets = pd.read_csv("data/Encoded_Tweets_Dataset.csv")
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tweet    1785 non-null   object
 1   Suicide  1785 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 28.0+ KB


In [3]:
tweets.head(5)

,Tweet,Suicide
0,making some lunch,0
1,@Alexia You want his money.,0
2,@dizzyhrvy that crap took me forever to put to...,1
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,0
4,Trying out &quot;Delicious Library 2&quot; wit...,0


## Set Variables

Bagian ini menyatakan variabel yang akan digunakan untuk path pada directory yang akan digunakan untuk mengakses file yang dibutuhkan atau menyimpan file yang dihasilkan. Hal ini mencakup penamaan folder pipeline menyesuaikan kriteria pengumpulan proyek.

In [4]:
PIPELINE_NAME = "nkumala16-pipeline"
SCHEMA_PIPELINE_NAME = "suicide-tweet-detection-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

Kemudian, juga dibuat variabel untuk mengakses sumebr data, yaitu pada folder 'data' dan object InteractiveContext() yang akan digunakan dalam projek.

In [5]:
DATA_ROOT = "data"

In [6]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

Berikut merupakan kompnen pertama dari ML pipeline yang dibutuhkan untuk proses Data Ingestion, CsvExampleGen() yang dipilih karena sumber dataset memiliki format CSV. Komponen ini telah disediakan pada library TensorFLow Extension (TFX).

Pada bagian ini, dataset dibagi menjadi dua untuk proses 'train' dan 'eval' dengan rasio 8:2 yang diatur melalui parameter hash_buckets.

In [7]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Kode berikut digunakan untuk melihat dan menjalankan komponen ExampleGen() secara interaktif menggunakan object InteractiveCOntext() yang telah dibuat pada bagian sebelumnya.

In [8]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 103
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

Bagian ini akan menjelaskan pembuatan komponen ML pipeline untuk validasi data yang terdiri dari tiga tahap:
1. Statistics Summary
2. Data Schema
3. Anomaly Detection

### Statistics Summary

Berikut merupakan tahapan pertama dari ML pipeline yang dibutuhkan untuk proses Data Validation, pembuatan rangkuman statistik. Proses ini dapat dilakukan dengan menggunakan komponen StatisticGen() pada TFX yang memilikiparameter input 'examples' untuk menerima dataset dari komponen ExampleGen pada tahapan sebelumnya.

In [9]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 104
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Data Schema

Tahapn kedua dari proses validasi data adalah dengan membuat skema data menggunakan komponen SchemaGen() dengan input berupa ringkasan statistik yang didapatkan melalui tahapan sebelumnya.

In [11]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
    )
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 105
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

Berdasarkan output dari kode berikut, diketahui bahwa terdapat dua fitur dataset yang digunakan, yaitu 'Suicide' dengan tipe data integer untuk status makna postingan yang tersimpan pada fitur 'Tweet' dengan tipe data bytes.

In [12]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Suicide',INT,required,,-
'Tweet',BYTES,required,,-


### Anomaly Detection

Bagian terakhir dari proses data validation, identifikasi apakah terdapat anomali pada dataset yang digunakan untuk pembuatan ML pipeline. Tahapn ini dilakukan dengan menggunakan komponen ExampleValidator() yang mengambil parameter input dari hasil tahapan statistics_gen dan schema-gen.

In [13]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 106
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [14]:
interactive_context.show(example_validator.outputs['anomalies'])

Tidak ditemukan anomali pada dataset.

## Data Preprocessing

Tahapan selanjutnya adalah praproses data dengan mengubah data mentah menjadi data yang siap digunakan untuk training model. Tahapan ini dapat dilakukan dengan menggunakan komponen Transform dan TensorFlow Transform (TFT). Input untuk tahapan ini berupa modul file yang berisi fungsi preprocessing dengan menggunakan command '%%writefile' untuk membuat atau overwrite sebuah file dan mengisinya dengan kumpulan statement yang ada pada baris-baris selanjutnya.

In [15]:
TRANSFORM_MODULE_FILE = "suicide-tweets_transform.py"

Kode berikut akan menghasilkan sebuah modul dengan format python (.py) dengan nama 'suicide-tweets_transform.py' sesuai dengan yang sudah ditentukan pada kode sebelumnya.

In [16]:
%%writefile suicide-tweets_transform.py

import tensorflow_transform as tft
import tensorflow as tf

LABEL_KEY = "Suicide"
FEATURE_KEY = "Tweet"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    outputs = {}
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    return outputs

Overwriting suicide-tweets_transform.py


In [17]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\nkumala16-pipeline\Transform\transform_graph\107\.temp_path\tftransform_tmp\711006bb0fd343b9b61befe3dd0b9567\assets


INFO:tensorflow:Assets written to: pipelines\nkumala16-pipeline\Transform\transform_graph\107\.temp_path\tftransform_tmp\711006bb0fd343b9b61befe3dd0b9567\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 107
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Development & Validation

Bagian ini akan menjelaskan pembuatan komponen ML pipeline untuk pengembangan dan validasi model yang terdiri dari tiga tahap:
1. Trainer
2. Resolver
3. Evaluator

### Trainer

In [18]:
TRAINER_MODULE_FILE = "suicide-tweets_trainer.py"

Kode berikut akan menghasilkan modul dengan nama 'suicide-tweets_trainer.py' sesuai dengan nama yang telah didefinisikan sebelumnya. Komponen Trainer() yang akan didefiniskan setelahnya memiliki beberapa input, seperti module_file, examples, schema, transform_graph, train_args, dan eval_args. Tampilan detail kemudian dapat dilihat dengan menjalankan interactive_context pada bagian bawah kode.

In [19]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "Suicide"
FEATURE_KEY = "Tweet"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset
 
# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting suicide-tweets_trainer.py


In [20]:
from tfx.proto import trainer_pb2
 
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Tweet_xf (InputLayer)       [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                1088  


Epoch 1: val_binary_accuracy improved from -inf to 0.88924, saving model to pipelines\nkumala16-pipeline\Trainer\model\108\Format-Serving


INFO:tensorflow:Assets written to: pipelines\nkumala16-pipeline\Trainer\model\108\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\nkumala16-pipeline\Trainer\model\108\Format-Serving\assets


1000/1000 [==============================] - 9s 7ms/step - loss: 0.2095 - binary_accuracy: 0.9061 - val_loss: 0.3606 - val_binary_accuracy: 0.8892
INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\nkumala16-pipeline\Trainer\model\108\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\nkumala16-pipeline\Trainer\model\108\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 108
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Resolver

Termasuk dalam bagian analisis dan validasi model, komponen Resolver() berfungsi untuk menyediakan baseline model yang merupakan hal penting ketika kita akan memiliki lebih dari satu versi model dan ingin membandingkan kedua versi tersebut.

In [21]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 109
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluator

Tahapan selanjutnya adalah membuat konfigurasi yang berfungsi untuk mengevaluasi model menggunakan library TFMA.

In [26]:
import tensorflow_model_analysis as tfma 

# LABEL_KEY = "Suicide"
# FEATURE_KEY = "Tweet"

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Suicide')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

Selanjutnya, dibuatlah komponen Evaluator() yang akan menerima beberapa input, seperti examples, model, baseline_model, dan eval_config.

In [27]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config) # type: ignore
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 112
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Kode berikut akan menampilkan tampilan interaktif untuk hasil evaluasi dari komponen Evaluator() dengan menggunakan library TFMA.

In [28]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result) # type: ignore
tfma.view.render_slicing_metrics(tfma_result) # type: ignore
tfma.addons.fairness.view.widget_view.render_fairness_indicator( # type: ignore
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Pusher

Bagian akhir dari ML pipelibe model ini adalah komponen Pusher() yang akan menerima input berupa trained_model, hasil evaluasi, dan argumen terkait serving file path.

In [29]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/suicide-detection-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 113
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))